In [2]:
!pip install huggingface_hub llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422320 sha256=7ac700d97f8607a88543726b479ba93a497a2a0ad8bb6cda90ea00df5160537f
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [5]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# Make sure your team runs `huggingface-cli login` in their terminal first!

# 1. Automatically fetch the model from your Private Hugging Face Repo
# (It downloads once and caches it locally on their machine)
print("📥 Fetching Night Sentinel from Hugging Face...")
model_path = hf_hub_download(
    repo_id="Ismailea04/medgemma-night-sentinel", # Replace with your repo
    filename="medgemma-night-sentinel-Q4_K_M.gguf"
)

# 2. Load the GGUF model into local RAM/CPU
print("🧠 Booting Clinical Brain...")
llm = Llama(
    model_path=model_path,
    n_ctx=2048,       # Memory window for the prompt
    verbose=False     # Set to True if you want to see C++ debug logs
)

# 3. The Function Saad can use for his Agent
def trigger_night_sentinel(patient_profile, vital_signs_anomaly):
    """
    Feeds an anomaly to the steered MedGemma model.
    """
    # Using the strict MedGemma Instruction-Tuning Template
    prompt = (
        f"<start_of_turn>user\n"
        f"[NIGHT SENTINEL SYSTEM]\n"
        f"Patient Profile: {patient_profile}\n"
        f"Event/Anomaly: {vital_signs_anomaly}\n"
        f"    TASK:\n"
        f"1. Compare the current data to the patient's specific baseline (Name it : ##Comparaison).\n"
        f"2. Identify any clinical anomalies (Name it : ##Detection).\n"
        f"3. Provide a short clinical interpretation of what might be happening (Name it : ##Interpretation).<end_of_turn>\n"
        f"<start_of_turn>model\n"
    )

    response = llm(
        prompt,
        max_tokens=400,
        stop=["<end_of_turn>"], # Stop generating when the thought is finished
        echo=False
    )

    return response['choices'][0]['text'].strip()



📥 Fetching Night Sentinel from Hugging Face...
🧠 Booting Clinical Brain...


llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_unified_iswa: using full-size SWA cache (ref: https://github.com/ggml-org/llama.cpp/pull/13194#issuecomment-2868343055)


In [6]:
# --- TEST EXECUTION ---
if __name__ == "__main__":
    profile = "65yo male, post-operative recovery. Baseline HR 70."
    anomaly = "Sudden HR spike to 135 bpm with irregular rhythm, SpO2 92%."

    print("\n🚨 Triggering Alert Analysis...")
    diagnosis = trigger_night_sentinel(profile, anomaly)
    print(f"\n[SENTINEL REPORT]\n{diagnosis}")


🚨 Triggering Alert Analysis...

[SENTINEL REPORT]
Here's an analysis of the provided data, broken down into the requested sections:

##Comparison

*   **Baseline HR:** 70 bpm
*   **Current HR:** 135 bpm
*   **Current Rhythm:** Irregular
*   **SpO2:** 92%

##Detection

*   **Significant HR Spike:** The patient's heart rate has increased from 70 bpm to 135 bpm.
*   **Irregular Rhythm:** The heart rate is no longer regular.
*   **Hypoxia:** The patient's oxygen saturation is 92%, which is below the typical normal range of 95-100% and indicates a degree of hypoxemia.

##Interpretation

The patient is experiencing a sudden and significant increase in heart rate with an irregular rhythm, accompanied by a drop in oxygen saturation. This suggests a possible cardiac event (e.g., arrhythmia, possibly atrial fibrillation or other irregular rhythm), and/or a respiratory issue. The irregular rhythm could be concerning, and the hypoxia needs to be investigated. Given the post-operative status, othe